In [ ]:
# all dependencies can be found in requirements.txt
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar, Calendar, Tab
import pyecharts

In [ ]:
df = pd.read_excel(
    io="Financial_Data.xlsx",
    engine="openpyxl",
    sheet_name="Orders",
    skiprows=2,
    usecols="B:T",
    nrows=3312,)
df

In [ ]:
df.info()

### Manipulating the Data to Graph *Monthly Sales*

In [ ]:
df["Month"] = df["Order Date"].dt.month
df

In [ ]:
group_by_months = df.groupby(by=["Month"]).sum()[["Sales", "Profit"]]
group_by_months

### *Note*: Pyecharts does not accept Pandas dataframe
### Therefore, all dataframes columns are converted to arrays

In [ ]:
monthly_sales = (
    Bar()
    .add_xaxis(group_by_months.index.tolist())
    .add_yaxis("Sales", group_by_months["Sales"].round(0).tolist())
    .add_yaxis("Profit", group_by_months["Profit"].round(0).tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sales & Profit by Month", subtitle="in USD")
    )
)

## Sales & Profit by Month Bar Plot

In [ ]:
monthly_sales.render_notebook()

## Sales & Profit by Sub-Category Bar Plot

In [ ]:
group_by_subcat = df.groupby(by=["Sub-Category"], as_index=False).sum().sort_values(by=["Sales"])
group_by_subcat

In [ ]:
sales_and_profit_by_subcat = (
    Bar()
    .add_xaxis(group_by_subcat.index.tolist())
    .add_yaxis("Sales", group_by_subcat["Sales"].round(0).tolist())
    .add_yaxis("Profit", group_by_subcat["Profit"].round(0).tolist())
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sales & Profit by Sub-Category", subtitle="in USD"),
    )
)

In [ ]:
sales_and_profit_by_subcat.render_notebook()

## Sales & Profit by Sub-Category Bar Plot

In [ ]:
sales_by_date = df.groupby(by=["Order Date"]).sum()[["Sales"]].round(0)
sales_by_date = sales_by_date.reset_index()
sales_by_date

## Sales Calendar

In [ ]:
# index needs to be reset in order for Pandas to register the "Order Date" column
sales_by_date_data = sales_by_date[["Order Date", "Sales"]].values.tolist()
max_sales = df["Sales"].max()
min_sales = df["Sales"].min()

In [ ]:
sales_calendar = (
    Calendar()
    .add("", sales_by_date_data, calendar_opts=opts.CalendarOpts(range_="2021"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sales Calendar", subtitle="in USD"),
        legend_opts=opts.LegendOpts(is_show=False),
        visualmap_opts=opts.VisualMapOpts(
            max_= max_sales,
            min_= min_sales,
            orient="horizontal",
            is_piecewise=False,
            pos_top="230px",
            pos_left="100px"
        )
    )
)

### Drag the Multicolor Slider below to Filter Sales Amount Displayed

In [ ]:
sales_calendar.render_notebook()

## Compile All Charts into HTML File

In [ ]:
tab = Tab(page_title="Sales & Profit Overview")
tab.add(monthly_sales, "Sales & Profit by Month")
tab.add(sales_and_profit_by_subcat, "Sales & Profit by Sub-Category")
tab.add(sales_calendar, "Sales Calendar")
tab.render("Sales_and_Profit_Overview.html")